# Visualize Evaluation Run

In [ ]:
# @title Configuration
# fmt: off
GOOGLE_CLOUD_PROJECT = "" # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
GOOGLE_CLOUD_REGION = "" # @param {type: "string", placeholder: "[your-region]", isTemplate: true}
EXPERIMENT_NAME = "agent-evaluation-researcher" # @param {type: "string", placeholder: "[experiment name]", isTemplate: true}
RUN_NAME = "(latest)" # @param {type: "string", placeholder: "[run name]", isTemplate: true}
# fmt: on

## Dependencies

In [ ]:
%pip install google-cloud-aiplatform[evaluation]==1.133.* vertexai google-genai==1.59.* -q

## Authentication for Google Colab

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Initialize

In [ ]:
import os
from google.cloud import aiplatform
from dotenv import load_dotenv

load_dotenv()

if not GOOGLE_CLOUD_PROJECT:
    GOOGLE_CLOUD_PROJECT = os.getenv("GOOGLE_CLOUD_PROJECT")

if not GOOGLE_CLOUD_REGION:
    GOOGLE_CLOUD_REGION = os.getenv(
        "GOOGLE_CLOUD_REGION",
        os.getenv(
            "GOOGLE_CLOUD_LOCATION",
        ),
)

project_id = GOOGLE_CLOUD_PROJECT
location = GOOGLE_CLOUD_REGION
if not location or location == "global":
    location = !gcloud config get-value compute/region -q 2>/dev/null
    if location:
        location = location[0]
    else:
        location = "us-central1"

aiplatform.init(project=project_id, location=location)

## Retrieve Evaluation Run

In [ ]:
import json
from google.cloud import aiplatform
from google.cloud.storage import Blob, Client as StorageClient
from vertexai import types # type: ignore

import pandas as pd

storage_client = StorageClient(project=project_id)

experiment = aiplatform.Experiment.get(
    experiment_name=EXPERIMENT_NAME,
    project=project_id,
    location=location
)

if RUN_NAME == "(latest)":
    runs = aiplatform.ExperimentRun.list(
        experiment=experiment,
        project=project_id,
        location=location
    )
    run_name = runs[0].name
else:
    run_name = RUN_NAME
experiment_run =aiplatform.ExperimentRun.get(
    experiment=experiment,
    run_name=run_name,
    project=project_id,
    location=location
)
experiment_parameters = experiment_run.get_params()
eval_results = Blob.from_uri(
    experiment_parameters["evaluation_results"],
    client=storage_client
).download_as_text()
eval_data_jsonl_bytes = Blob.from_uri(
    experiment_parameters["evaluation_data"],
    client=storage_client
).download_as_bytes()
eval_run_result = types.EvaluationResult.model_validate_json(eval_results)
data = []
for line in eval_data_jsonl_bytes.splitlines():
    data.append(json.loads(line.decode("utf-8")))
# Replace GCS source with DataFrame for display
eval_run_result.evaluation_dataset[0].gcs_source = None
eval_run_result.evaluation_dataset[0].eval_dataset_df = pd.DataFrame(data)

## Show Evaluation Report

In [ ]:
eval_run_result.show()